# Einstein-Hilbert action
Let us derive the Einstein Field Equations, both in the absense and prescence of matter, from modified Einstein-Hilbert action.

## Definitions
We begin with some precusory definitions:

In [1]:
{a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p ,q, r, s, t, u#}::Indices(position=independent).
g_{a b}::Metric.
g^{a b}::InverseMetric.
{g^{a}_{b}, g_{a}^{b}}::KroneckerDelta.

\dg{#}::LaTeXForm("\delta g").

We use `::LaTeXForm` for variation variables, as performing substitutions and collections on a single tensor is easier. Furthermore, were these equations to be evaluated, it is benefitial to not use derivative terms on the LHS, as Cadabra support for such situations is limited.

We define the derivative terms and symmetries of the Riemann connections:

In [2]:
\partial{#}::PartialDerivative.
{\nabla{#}, \delta{#}}::Derivative.

 # don't use spaces in the indices array
\Gamma^{a}_{b c}::TableauSymmetry(shape={2}, indices={1,2}).

\dGamma{#}::LaTeXForm("\delta \Gamma").
\dGamma^{a}_{b c}::TableauSymmetry(shape={2}, indices={1,2}).

We then define the Riemann and Ricci tensors, and the Ricci scalar:

In [3]:
Rabcd := R^{a}_{b c d} = \partial_{c}{\Gamma^{a}_{b d}}
    - \partial_{d}{\Gamma^{a}_{b c}}
    + \Gamma^{e}_{b d} \Gamma^{a}_{c e}
    - \Gamma^{e}_{b c} \Gamma^{a}_{d e};

Rab := R_{a b} = R^{f}_{a f b};

Rscalar := R = g^{a b}R_{a b};

${}R^{a}\,_{b c d} = \partial_{c}{\Gamma^{a}\,_{b d}}-\partial_{d}{\Gamma^{a}\,_{b c}}+\Gamma^{e}\,_{b d} \Gamma^{a}\,_{c e}-\Gamma^{e}\,_{b c} \Gamma^{a}\,_{d e}$

${}R_{a b} = R^{f}\,_{a f b}$

${}R = g^{a b} R_{a b}$

## Varying compontents
To compute the overal variation of an action, we need to know how the constituent components change.

### Variation of the metric:
This result follows from [Jacobi's formula](https://en.wikipedia.org/wiki/Jacobi%27s_formula), which, given as a result for the metric, is easier just to hard code than rederive.

We also include short hands for the determinant of the metric, and the variation of the determinant.

In [4]:
\detm::LaTeXForm("\sqrt{\left| g \right|}").
\ddetm::LaTeXForm("\delta \sqrt{\left| g \right|}").
vmetric := \ddetm = \frac{-1}{2} \detm g_{a b} \dg^{a b});

${}\delta \sqrt{\left| g \right|} =  - \frac{1}{2}\sqrt{\left| g \right|} g_{a b} \delta g^{a b}$

### Variation of the Ricci Scalar
We begin by calculating the variation of the Riemann tensor, with the intention of contracting the result:

In [5]:
expr := @(Rabcd);

\dR{#}::LaTeXForm("\delta R").

variations := {
    R^{a}_{b c d} -> \dR^{a}_{b c d}, 
    \Gamma^{a}_{b c} -> \dGamma^{a}_{b c}
}.
vary(expr, variations);

${}R^{a}\,_{b c d} = \partial_{c}{\Gamma^{a}\,_{b d}}-\partial_{d}{\Gamma^{a}\,_{b c}}+\Gamma^{e}\,_{b d} \Gamma^{a}\,_{c e}-\Gamma^{e}\,_{b c} \Gamma^{a}\,_{d e}$

${}\delta R^{a}\,_{b c d} = \partial_{c}{\delta \Gamma^{a}\,_{b d}}-\partial_{d}{\delta \Gamma^{a}\,_{b c}}+\delta \Gamma^{e}\,_{b d} \Gamma^{a}\,_{c e}+\Gamma^{e}\,_{b d} \delta \Gamma^{a}\,_{c e}-\delta \Gamma^{e}\,_{b c} \Gamma^{a}\,_{d e}-\Gamma^{e}\,_{b c} \delta \Gamma^{a}\,_{d e}$

Next we will calculate the covariant derivative of the connections. We can already begin to see in the variation of the Riemann tensor that terms constructed from the covariant derivative exist, and all we wish to do is substitute where appropriate.

With help of the `cdb.core.manip` library, we can manipulate equations in a more conventional sense -- we will use this to balance and multiply by terms.

In [6]:
import cdb.core.manip as manip

DGamma := \nabla_{d}(\dGamma^{a}_{b c}) =
    \partial_{d}{\dGamma^{a}_{b c}}
    + \Gamma^{a}_{d e}\dGamma^{e}_{b c}
    - \Gamma^{e}_{b d}\dGamma^{a}_{e c}
    - \Gamma^{e}_{d c}\dGamma^{a}_{b e};
    
DGamma = manip.to_rhs(DGamma, $\nabla_{d}(A??)$)
DGamma = manip.to_lhs(DGamma, $\partial_{d}(A??)$)
DGamma = manip.multiply_through(DGamma, $-1$);

${}\nabla_{d}{\delta \Gamma^{a}\,_{b c}} = \partial_{d}{\delta \Gamma^{a}\,_{b c}}+\Gamma^{a}\,_{d e} \delta \Gamma^{e}\,_{b c}-\Gamma^{e}\,_{b d} \delta \Gamma^{a}\,_{e c}-\Gamma^{e}\,_{d c} \delta \Gamma^{a}\,_{b e}$

${}\partial_{d}{\delta \Gamma^{a}\,_{b c}} = -\Gamma^{a}\,_{d e} \delta \Gamma^{e}\,_{b c}+\Gamma^{e}\,_{b d} \delta \Gamma^{a}\,_{e c}+\Gamma^{e}\,_{d c} \delta \Gamma^{a}\,_{b e}+\nabla_{d}{\delta \Gamma^{a}\,_{b c}}$

We use `sort_product` to ensure that the expression is sorted before we perform any substitutions: if this is omitted, Cadabra may not be able to identify all symmetries in the `meld` expression. 

A general *rule-of-thumb* is to call `sort_product` before performing a simplfication in general, as it only rearranges the expression and performs no other change.

In [7]:
sort_product(expr);
substitute(expr, DGamma);
meld(expr);

${}\delta R^{a}\,_{b c d} = \partial_{c}{\delta \Gamma^{a}\,_{b d}}-\partial_{d}{\delta \Gamma^{a}\,_{b c}}+\Gamma^{a}\,_{c e} \delta \Gamma^{e}\,_{b d}+\Gamma^{e}\,_{b d} \delta \Gamma^{a}\,_{c e}-\Gamma^{a}\,_{d e} \delta \Gamma^{e}\,_{b c}-\Gamma^{e}\,_{b c} \delta \Gamma^{a}\,_{d e}$

${}\delta R^{a}\,_{b c d} = \Gamma^{e}\,_{b c} \delta \Gamma^{a}\,_{e d}+\Gamma^{e}\,_{c d} \delta \Gamma^{a}\,_{b e}+\nabla_{c}{\delta \Gamma^{a}\,_{b d}}-\Gamma^{e}\,_{b d} \delta \Gamma^{a}\,_{e c}-\Gamma^{e}\,_{d c} \delta \Gamma^{a}\,_{b e}-\nabla_{d}{\delta \Gamma^{a}\,_{b c}}+\Gamma^{e}\,_{b d} \delta \Gamma^{a}\,_{c e}-\Gamma^{e}\,_{b c} \delta \Gamma^{a}\,_{d e}$

${}\delta R^{a}\,_{b c d} = \nabla_{c}{\delta \Gamma^{a}\,_{b d}}-\nabla_{d}{\delta \Gamma^{a}\,_{b c}}$

Now we find the so-called Palatini identity by substituting the expression in the contraction for the Ricci scalar, varied on both sides:

In [8]:
palatini := \dR_{c b} = \dR^{a}_{c a b};
substitute(palatini, expr);

${}\delta R_{c b} = \delta R^{a}\,_{c a b}$

${}\delta R_{c b} = \nabla_{a}{\delta \Gamma^{a}\,_{c b}}-\nabla_{b}{\delta \Gamma^{a}\,_{c a}}$

### Scalar variation
We now wish to vary the Ricci scalar, for which we apply much the same prescription:

In [9]:
scalar := @(Rscalar).
variations := {
    R -> \dR,
    R_{a b} -> \dR_{a b},
    g^{a b} -> \dg^{a b}
}.
vary(scalar, variations);
substitute(scalar, palatini);
distribute(scalar)
sort_product(scalar);

${}\delta R = \delta g^{a b} R_{a b}+g^{a b} \delta R_{a b}$

${}\delta R = \delta g^{a b} R_{a b}+g^{a b} \left(\nabla_{c}{\delta \Gamma^{c}\,_{a b}}-\nabla_{b}{\delta \Gamma^{c}\,_{a c}}\right)$

${}\delta R = R_{a b} \delta g^{a b}+\nabla_{c}{\delta \Gamma^{c}\,_{a b}} g^{a b}-\nabla_{b}{\delta \Gamma^{c}\,_{a c}} g^{a b}$

We make a substitution, using the general result that $\nabla_c g^{a b} = 0$:

In [10]:
covG := \nabla_{a}{A??}g^{b c} -> \nabla_{a}(g^{b c}A??).
substitute(scalar, covG);

${}\delta R = R_{a b} \delta g^{a b}+\nabla_{c}\left(g^{a b} \delta \Gamma^{c}\,_{a b}\right)-\nabla_{b}\left(g^{a b} \delta \Gamma^{c}\,_{a c}\right)$

## Vacuum action
We use the vacuum Einstein-Hilbert action in the absense of any additional fields or cosmological constant:

In [11]:
action := S = \int{\detm R}{x};

\dS::LaTeXForm("\delta S").

variations := {
    S -> \dS,
    R -> \dR,
    \detm -> \ddetm
}.
vary(action, variations);

substitute(action, vmetric);
substitute(action, scalar);

distribute(action)
rename_dummies(action)
factor_out(action, $\dg^{a b}, \detm$);

${}S = \int \sqrt{\left| g \right|} R\,\,{\rm d}x$

${}\delta S = \int \left(\delta \sqrt{\left| g \right|} R+\sqrt{\left| g \right|} \delta R\right)\,\,{\rm d}x$

${}\delta S = \int \left( - \frac{1}{2}\sqrt{\left| g \right|} g_{a b} \delta g^{a b} R+\sqrt{\left| g \right|} \delta R\right)\,\,{\rm d}x$

${}\delta S = \int \left( - \frac{1}{2}\sqrt{\left| g \right|} g_{a b} \delta g^{a b} R+\sqrt{\left| g \right|} \left(R_{a b} \delta g^{a b}+\nabla_{c}\left(g^{a b} \delta \Gamma^{c}\,_{a b}\right)-\nabla_{b}\left(g^{a b} \delta \Gamma^{c}\,_{a c}\right)\right)\right)\,\,{\rm d}x$

${}\delta S = \int \left(\sqrt{\left| g \right|} \delta g^{a b} \left( - \frac{1}{2}g_{a b} R+R_{a b}\right)+\sqrt{\left| g \right|} \left(\nabla_{a}\left(g^{b c} \delta \Gamma^{a}\,_{b c}\right)-\nabla_{c}\left(g^{b c} \delta \Gamma^{a}\,_{b a}\right)\right)\right)\,\,{\rm d}x$

Terms proportional to $\delta g^{a b}$ are the Vacuum Einstein Field Equations, the rest is a total derivative.

We select the component to rewrite it in the conventional sense:

In [12]:
vac = action[1][0][0][2]
vac := 0 = @(vac);

${}0 =  - \frac{1}{2}g_{a b} R+R_{a b}$

And show that $R=0$ by contraction with the metric:

In [13]:
vac2 := @(vac).
manip.multiply_through(vac2, $g^{a b}$)
distribute(vac2);
eliminate_metric(vac2);
substitute(vac2, $R^{a}_{a} -> R, g^{a}_{a} -> 4$);
vac2 = manip.swap_sides(vac2)
vac2 = manip.multiply_through(vac2, $-1$);

${}0 =  - \frac{1}{2}g^{a b} g_{a b} R+g^{a b} R_{a b}$

${}0 =  - \frac{1}{2}g^{b}\,_{b} R+R^{b}\,_{b}$

${}0 = -R$

${}R = 0$

We can now state the equations in the canonical form:

In [14]:
substitute(vac, vac2)
vac = manip.swap_sides(vac);

${}R_{a b} = 0$

## Adding matter field
We modify our action to include a matter field, and the Einstein gravitational constant $\kappa = 8\pi G$:

In [15]:
\matter::LaTeXForm("\mathcal{L}_{\text{mat}}").
\dmatter::LaTeXForm("\delta \mathcal{L}_{\text{mat}}").
action := S = \int{\detm ( \frac{1}{2 \kappa} R + \matter ) }{x};

${}S = \int \sqrt{\left| g \right|} \left(\frac{1}{2}{\kappa}^{-1} R+\mathcal{L}_{\text{mat}}\right)\,\,{\rm d}x$

### Energy-Momentum tensor
This is another result which is easier to hardcode, and the modify for our needs:

In [16]:
emTensor := \detm T_{a b} \dg^{a b} = -2 \delta(\detm \matter);
product_rule(emTensor)
distribute(emTensor);

rename := {
    \delta(\detm) -> \ddetm, 
    \delta(\matter) -> \dmatter
}.
substitute(emTensor, rename)
substitute(emTensor, vmetric)
emTensor = manip.swap_sides(emTensor);

${}\sqrt{\left| g \right|} T_{a b} \delta g^{a b} = -2\delta\left(\sqrt{\left| g \right|} \mathcal{L}_{\text{mat}}\right)$

${}\sqrt{\left| g \right|} T_{a b} \delta g^{a b} = -2\delta{\sqrt{\left| g \right|}} \mathcal{L}_{\text{mat}}-2\sqrt{\left| g \right|} \delta{\mathcal{L}_{\text{mat}}}$

${}\sqrt{\left| g \right|} g_{a b} \delta g^{a b} \mathcal{L}_{\text{mat}}-2\sqrt{\left| g \right|} \delta \mathcal{L}_{\text{mat}} = \sqrt{\left| g \right|} T_{a b} \delta g^{a b}$

Using the identical prescription as in the vacuum case, we now vary the action and substitute terms:

In [17]:
action := S = \int{\detm ( \frac{1}{2 \kappa} R + \matter ) }{x};

variations := {
    S -> \dS,
    R -> \dR,
    \detm -> \ddetm,
    \matter -> \dmatter
}.
vary(action, variations);

substitute(action, vmetric)
substitute(action, scalar)
distribute(action)
substitute(action, emTensor)

distribute(action)
rename_dummies(action)
factor_out(action, $\dg^{a b}, \detm$);

${}S = \int \sqrt{\left| g \right|} \left(\frac{1}{2}{\kappa}^{-1} R+\mathcal{L}_{\text{mat}}\right)\,\,{\rm d}x$

${}\delta S = \int \left(\delta \sqrt{\left| g \right|} \left(\frac{1}{2}{\kappa}^{-1} R+\mathcal{L}_{\text{mat}}\right)+\sqrt{\left| g \right|} \left(\frac{1}{2}{\kappa}^{-1} \delta R+\delta \mathcal{L}_{\text{mat}}\right)\right)\,\,{\rm d}x$

${}\delta S = \int \left(\sqrt{\left| g \right|} \delta g^{a b} \left( - \frac{1}{4}g_{a b} {\kappa}^{-1} R - \frac{1}{2}T_{a b}+\frac{1}{2}{\kappa}^{-1} R_{a b}\right)+\sqrt{\left| g \right|} \left(\frac{1}{2}{\kappa}^{-1} \nabla_{a}\left(g^{b c} \delta \Gamma^{a}\,_{b c}\right) - \frac{1}{2}{\kappa}^{-1} \nabla_{c}\left(g^{b c} \delta \Gamma^{a}\,_{b a}\right)\right)\right)\,\,{\rm d}x$

Select the component we are interested in, namely mulitples of $\delta g^{a b}$, and use the definition 
$$
G_{a b} = R_{a b} - \frac{1}{2}g_{a b} R.
$$

In [18]:
efe = action[1][0][0][2]
efe := 0 = 2 \kappa @(efe).
distribute(efe)
collect_factors(efe);
substitute(efe, $R_{a b} - \frac{1}{2}g_{a b} R -> G_{a b}$);
manip.to_lhs(efe, $G_{a b}$)
manip.multiply_through(efe, $-1$);

${}0 =  - \frac{1}{2}g_{a b} R-\kappa T_{a b}+R_{a b}$

${}0 = -\kappa T_{a b}+G_{a b}$

${}G_{a b} = \kappa T_{a b}$